# Exercise Sheet No. 3

---

> Machine Learning for Natural Sciences, Summer 2021, Jun.-Prof. Pascal Friederich, pascal.friederich@kit.edu
> 
> Deadline: 09.05.2021, 8am

---

**Topic**: This exercise sheet will focus on linear algebra, precision, recall and feature reduction. You will continue to use ``numpy`` methods.

## 3.1 Distance function computation

For many feature descriptors and also for a specific implementation of the nearest neighbor classifier, the Euclidean mutual distance between a set of points must be computed. This serves as a measure on how two points are distinct or how far apart they are in Euclidean space. We will make use of this very often in the future. Therefore this exercise focus on an efficient implementation using numpy. You should gain a better unstanding of `numpy` methods.

**3.1.1** Implement a function ``dist_loop(A,B)`` to computes the Euclidean distance between all instances between two sets of points $A \subset \mathbb{R}^{D}$ and $B \subset \mathbb{R}^{D}$. The input should be two matrices of shape $N \times D$ and $M \times D$. Do this by using explicit python loops to find the distance $d_{ij} = || a_{i} - b_{j} ||$ between two points $a_{i} \in A$ and $b_{i} \in B$. The output should be a $N \times M$ distance matrix. For the calculation of the Euclidean distance you might want to use ``numpy.square()``, ``numpy.sum()`` and ``numpy.sqrt()``.

In [ ]:
import numpy as np
from numpy import random
import sys
n = 500
m = 1000
d = 3
A_data = np.reshape(random.rand(n*d),(n,d))
B_data = np.reshape(random.rand(m*d),(m,d))
print("A shape:", A_data.shape, "B shape:", B_data.shape)

In [ ]:
def dist_loop(A,B):
    if A.shape[-1] != B.shape[-1]:
        raise ValueError("Error A and B must have same last dimension but got", A.shape[-1], B.shape[-1])
    # YOUR CODE HERE
    raise NotImplementedError()

**3.1.2** Since loops are rather slow in python, and more efficient code is required for numeric operations, write another function ``dist_vec(A,B)`` to compute the distance relying
on vectorization with ``numpy`` methods. Consult https://www.safaribooksonline.com/library/view/python-for-data/9781449323592/ch04.html and https://softwareengineering.stackexchange.com/questions/254475/how-do-i-move-away-from-the-for-loop-school-of-thought for information on how to do this. 

Tip: There is more than one way to do it. For the most straightforward answer, you need to add an additional dimension to `np.arrays` via, for example, `expand_dims`. If two `np.arrays` do not have matching shape they are then automatically broadcasted by repeating their respective element along the axis in question. Beside broadcasting, also the [indexing rules](https://numpy.org/doc/stable/reference/arrays.indexing.html) (required for 3.2, 3.3) of `numpy` help with vectorization to avoid pyhton loops. You must not use ``scipy`` or its methods for this task!

In [ ]:
def dist_vec(A,B):
    # YOUR CODE HERE
    raise NotImplementedError()

Check that the two function return the same result. And check the output shape of the results.

In [ ]:
result_loop = dist_loop(A_data,B_data)
result_vec = dist_vec(A_data,B_data)
result_vec_shape = None # Check shape of result_loop
result_loop_shape = None # Check shape of result_vec
# YOUR CODE HERE
raise NotImplementedError()

Now compare the run times of the two implementations using jupyter's ``%timeit`` command or pythons ``time``. How much faster is the vectorized version?

In [ ]:
# measure time for loop
%timeit result_loop = dist_loop(A_data,B_data)
# measure time for loop
%timeit result_vec = dist_vec(A_data,B_data)

In [ ]:
speed_up_factor = 1 # A rough estimation is okay.
# YOUR CODE HERE
raise NotImplementedError()

Note: It is absolutely critical that you understand vectorization, and with this the indexing and broadcasting methods of numpy. Not only do we need fast performance, but also all deep learning frameworks are based on array or tensor operations just like you used with ``numpy``.

In [ ]:
# Test for grading
assert callable(dist_loop)

In [ ]:
assert callable(dist_vec)

In [ ]:
# Test for grading
assert result_loop is not None
assert result_vec is not None

In [ ]:
assert result_vec_shape is not None
assert result_loop_shape is not None

In [ ]:
# Test for grading
assert speed_up_factor > 0

## 3.2 Precision-Recall Curves

In this exercise, we use sklearn's digits dataset for an image retrieval experiment. Given any image
from this dataset as a query, your algorithm shall find all similar images, where we define similarity
by "contains the same digit". Of course, only the features (i.e. the pixel values of the images) may be
used for similarity search. The known labels only serve for testing the quality of the search result. This approach measures how clustered the calsses are in a defined feature space (i.e. pixels). Start loading the dataset.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

# read and prepare the digits data
digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
# Show digits
fig = plt.figure(figsize = (10,3))
plt.gray()
plt.subplot(1,3,1); plt.axis('off')
plt.imshow(images[0])
plt.subplot(1,3,2); plt.axis('off')
plt.imshow(images[1])
plt.subplot(1,3,3); plt.axis('off')
plt.imshow(images[2])
fig.tight_layout(); plt.show()
print(target[:3])

Define dissimilarity by the Euclidean distance between pixel values:

$d(X_i,X_{i'} ) = || X_i - X_{i'}||^2_2$

To efficiently compute these distances, you should use vectorization from exercise 3.1. Let $D$
be the full dissimilarity matrix, i.e. $D_{i i'} = d(X_i,X_{i'})$. An ``np.argsort()`` of row $D_i$ now gives the similarity ordering of all digits relative to query digit $X_i$. The response sets $S_{im}$ consist of the $m$ nearest instances to query $X_i$ (including $X_i$ itself), with $m$ running over all values from 1 to $N$.

Note: If you didn't manage to solve 3.1 you can now use `scipy.spatial.distance.cdist`. But you do not get points for this solution in 3.1.

In [ ]:
dist_xx = None # Compute distance matrix from data (data = digits["data"]), do not change the data here!
resp_Sim = None # Sort matrix dist_xx along each row, so that you can extract response sets for any m (with resp_Sim[row_idx, :m]) 
# YOUR CODE HERE
raise NotImplementedError()

The positive class is defined by the instances having the same label as the query, i.e. $Y_{i'} = Y_i$, and
$N_i = \# \{i' \in 1,\dots,N : Y_{i'} = Y_i \}$ is the total number of positives. Each response set defines a pair ($\text{precision}_{im}$; $\text{recall}_{im}$) as:

$\text{precision}_{im} = \frac{\text{TP}_i(m)}{\text{TP}_i(m) + \text{FP}_i(m)}$,      $\text{recall}_{im} = \frac{\text{TP}_i(m)}{N_i}$

where $\text{TP}_i(m) = \# \{i' \in S_{im} : Y_i = Y_{i'}\}$ and $\text{FP}_i(m) = \# \{i' \in S_{im} : Y_i \neq Y_{i'}\}$ are the number of true and false positives in $S_{im}$. Compute the $N\times N$ precision matrix $P$ and recall matrix $R$ whose elements are the precision (respectively recall) values from these pairs, i.e. $P_{im} = \text{precision}_{im}$ and $R_{im} = \text{recall}_{im}$. 

Tip: You can use broadcasting with `np.expand_dims(axis)` and `np.cumsum` for vectorization. You may use indexing and indexing via boolean arrays, too. 

In [ ]:
true_positives = None # as TP_i(m) of shape (1797, 1797)
false_positives = None #  as FP_i(m) of shape (1797, 1797)
total_positives = None # as N_i of shape (1797,)
# YOUR CODE HERE
raise NotImplementedError()
# Calculate the NxN precision and recall matrices
Pim = None # of shape (1797, 1797)
Rim = None # of shape (1797, 1797)
# YOUR CODE HERE
raise NotImplementedError()

For each digit class $k \in \{0,\dots,9\}$, compute $\bar{P}_k$ and $\bar{R}_k$ as the average of the rows of $P$ and $R$ referring to class $k$, i.e. where $Y_i = k$, with $m$ as the the free parameter.

In [ ]:
Pk_m = None # averaged Pim of shape (10,1797)
Rk_m = None # averaged Rim of shape (10,1797)
# YOUR CODE HERE
raise NotImplementedError()

Plot the resulting precision/recall curves (using $m$ as the free parameter) and determine the area-under-curve (AUC) for each $k$. Do not use the implementation of ``sklearn`` in this task. This means you have to plot Recall vs. Precision for each class.

In [ ]:
def auc_precision_recall(precision,recall):
    # precision is P(m) for a specific class of shape (1797,)
    # recall is R(m) for a specific class of shape (1797,)
    return np.abs(np.trapz(y=precision, x=(recall)))
# Get Area under curve with auc_precision_recall()
AUC = None # List of AUC for each number 0,...,9 via auc_precision_recall()

# Plot the Precision-Recall Curves
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test for grading
assert dist_xx is not None
assert dist_xx.shape[0] == 1797 and dist_xx.shape[1] == 1797

In [ ]:
assert resp_Sim is not None
try:
    assert np.all(resp_Sim[0:2,0:2] == np.array([[0,877],[1,93]]))
except:
    print("resp_Sim should contain indices that give a similarity ordering, please do not shuffle the data for grading tests.")

In [ ]:
# Test for grading
assert true_positives is not None
try:
    assert true_positives[0,0] == 1 and true_positives[0,-1] == 178
except:
    print("As a sanity test, the first entry should be always 1 and the lasts equals the number of class labels for that digit.")
    print("Number of 0's:", np.sum(target == 0))

In [ ]:
assert false_positives is not None
try:
    assert false_positives[0,0] == 0 and false_positives[0,-1] == 1619
except:
    print("As a sanity test, the first entry should be always 0 and the last equals the number of other class labels for that digit.")
    print("Number of classes;",len(target) ,"number of 0's:", np.sum(target == 0), "means: ", len(target) - np.sum(target == 0) )

In [ ]:
assert total_positives is not None

In [ ]:
# Test for grading
assert Pim is not None

In [ ]:
assert Rim is not None

In [ ]:
# Test for grading
assert Pk_m is not None
try:
    assert Pk_m[0,0] - 1 < 0.01 and Pk_m[0,-1] - 0.09905 < 0.01
except:
    print("For closest neighbours we expect high precision, and low precision for all neighbours considered")

In [ ]:
assert Rk_m is not None

In [ ]:
# Test for grading
assert AUC is not None
try:
    assert (AUC[0] - 0.9450) < 0.1 and (AUC[-1] - 0.4825) < 0.1
except:
    print("We expect a bad AUC for similar digits like 9,8,6. Interestingly '0' is well clustered with AUC of ca. 0.95")

### 3.3 PCA and LDA

Now we will try to reduce the number of features or pixels to two and re-test the precision/recall curves from exercise 3.2.

As a first approach we try an unsupervised reduction of features, namely via Principle Component Analysis (PCA). The task is to use `scikit-learn`'s implementation of the PCA to reduce the number of pixels to two and then to make a scatter plot of the new features, where the color of each point represents the digits number.  Which property should this scatterplot have in order for the new features to be especially suitable for similarity search?

In [ ]:
from sklearn.decomposition import PCA
new_features_pca = None # Make new features of shape (1797, 2)
# YOUR CODE HERE
raise NotImplementedError()

Repeat the experiment from above for new features but this time with a supervised feature reduction. We use the Linear Discriminant Analysis (LDA), which is linear too but seeks to fit class conditional densities to the data and using Bayes’ rule. You can use the `scikit-learn`'s implementation of LDA. Repeat the step above and make a scatter plot of the new features, where the color of each point represents the digits number. What is the difference? Which method is better suited for dissimilarity search?

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
new_features_lda = None # Make new features of shape (1797, 2)
# YOUR CODE HERE
raise NotImplementedError()

Now repeat steps of 3.2. To do so, simply wrap the solution of 3.2 in a function as indicated below.

Note: If you didn't solve 3.2, you can still give an answer, based on the scatter plots above, to the question in the last cell. The function below and the PCA/LDA recall-precision curve plots are not graded.

In [ ]:
def make_precision_recall_distance(data,target):
    Pk_m, Rk_m = None, None
    # YOUR CODE HERE
    raise NotImplementedError()
    return Pk_m, Rk_m

Now you can plot the precision-recall curves for the dissimilarity by the Euclidean distance of the new features for the PCA and LDA.

What do you observe? Does it improve with respect to the full digits? (manually set linear_reduction_answer to True or False to answer the question)

In [ ]:
linear_reduction_answer = None # Get better: True , get worse: False

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test for grading
assert new_features_pca is not None

In [ ]:
# Test for grading
assert new_features_lda is not None

In [ ]:
# Test for grading
assert isinstance(linear_reduction_answer,bool)